# MGMTMSA 408 Lecture 1: LP Duality

In this notebook we will explore the bakery example from class. 

## Building the model
Let us first load Gurobi.

In [1]:
from gurobipy import *

We will now create the model, variables, constraints and the objective.

In [2]:
# Create the model. 
m = Model()

# Add the variables. 
# addVar() will just add a single variable.
x_B = m.addVar()
x_C = m.addVar()
x_M = m.addVar()

# Create the constraints. 
# Resource constraints:
butter_constr = m.addConstr( 20*x_B + 100*x_C +  10*x_M  <= 26000)
flour_constr =  m.addConstr(100*x_B + 50 *x_C +  0 *x_M  <= 80000)
sugar_constr =  m.addConstr(  0*x_B + 50 *x_C + 150*x_M  <= 5000)

# Nonnegativity constraints:
# NB: we can add these explicitly. However, by default, a decision variable Gurobi is initialized
# with a lower bound of 0, so we don't need to add these.
# m.addConstr(x_B >= 0.0)
# m.addConstr(x_C >= 0.0)
# m.addConstr(x_M >= 0.0)

# Create the objective function.
m.setObjective(3*x_B + 4*x_C + 4.5*x_M, GRB.MAXIMIZE)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-25


That's all we need. The last thing we need to do is "update" the model; we must do this after we have finished making any changes to the model (e.g., adding variables/constraints or changing the objective).

In [3]:
m.update()

Finally, we can solve the model.

In [4]:
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3 rows, 3 columns and 7 nonzeros
Model fingerprint: 0xad1e1882
Coefficient statistics:
  Matrix range     [1e+01, 2e+02]
  Objective range  [3e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 8e+04]
Presolve time: 0.01s
Presolved: 3 rows, 3 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.1875000e+29   2.968750e+30   7.187500e-01      0s
       2    2.6500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.650000000e+03


The above output is the solution log of the Gurobi optimizer. 

The beginning of this log usually gives you some summary information about the problem.
For example, how many constraints (= rows), how many variables (= columns), how sparse is the constraint matrix.
The next piece of the output tells you the ranges of different values (the constraint matrix, the objective function coefficients, the variable bounds and the "right hand side", i.e., the constant terms in the constraints). We won't worry about this too much for this course (this is sometimes useful for diagnosing numerical issues).

The next piece tells you the information about presolving. Solvers like Gurobi will often do some clever pre-processing to remove variables whose values can be deduced before properly solving the problem. 

The part that is formatted like a table (Iteration Objective ... Time) shows the progress of the solver.

The last two lines indicate how long it took to solve, and the optimal objective. 

After we solve the problem, we will want to extract useful information about the problem. Here is some information we can extract.

In [5]:
# Extract the solution status. 
status = m.status
print(status)
if status == GRB.OPTIMAL:
    print("Solved to optimality")
    
# Extract the optimal values of the decision variables.
B_value = x_B.x
C_value = x_C.x
M_value = x_M.x

print("Number of buns: ", B_value)
print("Number of croissants: ", C_value)
print("Number of muffins: ", M_value)

# Extract the optimal objective value.
optimal_obj = m.objval
print("Optimal objective: ", optimal_obj)

2
Solved to optimality
Number of buns:  750.0
Number of croissants:  100.0
Number of muffins:  0.0
Optimal objective:  2650.0


So in summary:
- The problem was solved to optimality. 
- The optimal production plan is to produce 750 buns, 100 croissants, 0 muffins
- The total revenue will be $2650.0. 

## Formulating the dual problem

Let's now formulate the dual problem.


In [6]:
m_dual = Model()

p_B = m_dual.addVar()
p_F = m_dual.addVar()
p_S = m_dual.addVar()

# Create the dual constraints
m_dual.addConstr( 20*p_B + 100 *p_F >= 3 )
m_dual.addConstr( 100*p_B + 50 *p_F + 50 * p_S >= 4 )
m_dual.addConstr( 10*p_B + 150 * p_S >= 4.5 )

m_dual.setObjective( 26000 * p_B + 80000 * p_F + 5000 * p_S, GRB.MINIMIZE)

m_dual.update()

m_dual.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3 rows, 3 columns and 7 nonzeros
Model fingerprint: 0x07e6afa5
Coefficient statistics:
  Matrix range     [1e+01, 2e+02]
  Objective range  [5e+03, 8e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 4e+00]
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.156250e+00   0.000000e+00      0s
       2    2.6500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.650000000e+03


In [7]:
p_B_value = p_B.x
p_F_value = p_F.x
p_S_value = p_S.x

print("Dual variable for butter: ", p_B_value)
print("Dual variable for flour: ", p_F_value)
print("Dual variable for sugar: ", p_S_value)

# Extract the dual optimal objective value.
dual_optimal_obj = m_dual.objval
print("Dual optimal objective: ", dual_optimal_obj)

Dual variable for butter:  0.0
Dual variable for flour:  0.03
Dual variable for sugar:  0.05
Dual optimal objective:  2650.0


Notice that the optimal objective value of the dual problem is the same as the primal problem!!

## Accessing dual information

In some of the applications we will see, we will be interested in accessing the optimal dual variables or _shadow prices_ of the constraints. While we can solve the dual, we can actually access it directly from the primal problem. 

To access the dual variables, we can access the constraint objects we created before:

In [8]:
dual_values = [butter_constr.pi, flour_constr.pi, sugar_constr.pi]
print(dual_values)

[0.0, 0.03, 0.05]


The shadow prices tell us the marginal change in the objective function for a marginal change in the right hand side of the constraint. 

For example, the shadow price of the flour constraint is \\$0.03. Thus, if the total amount of flour in grams increases from 80,000 to 80,000 $+ \delta$, then the total revenue should increase by \\$ $0.03\delta$. 

An alternate interpretation is as follows: suppose we are approached by someone who offers to sell us more flour. That person charges us some unit price for the flour. The most we should pay for it, based on the shadow price, is \\$0.03 per gram.

The shadow price for the sugar constraint has a similar interpretation: the shadow price is \\$0.05, so we should not pay more than \\$0.05 per gram for any additional quantity of sugar. 

Notice that the shadow price for butter is zero. In other words, increasing the available butter will not change the optimal revenue. This makes sense; to see why, let us compute how much butter is being used in the optimal solution:

In [9]:
# Direct calculation of butter being used
20*B_value + 100 * C_value + 10 * M_value

25000.0

Notice that the total amount of butter used in our production plan is 25,000 g -- but we had 26,000 g available! 

If we are not using all of the available butter, then increasing the butter will not improve our revenue.

A different way to approach this is as follows. Instead of directly computing the quantity of butter used, we can compute the _slack_ of each constraint. The slack is the difference between the left and right hand sides of the constraints. In this production planning example, the slack is how much of each ingredient is left over. 

In [10]:
# Access the constraint slacks of the optimal solution.
slack_values = [butter_constr.slack, flour_constr.slack, sugar_constr.slack]
print(slack_values)

[1000.0, 0.0, 0.0]


**Question:** What do you notice about the slacks and the shadow prices? 